In [1]:
import sys
import os
# setting path
#sys.path.append('/work/scripts')
import pandas as pd
import seaborn as sns

In [2]:
import Sensitivity as sa
import output_utils as ou

# parameters influencing soil hydro/thermal state
### cmt_envground.txt
--------------------------------------------------------
#### tcsolid(m): Soil thermal conductivity for moss (W/mK)
current value: 0.25, range: 0.005-0.5

*Ekici et al. 2015,Jiang et al. 2015, O’Donnell et al. 2009*

#### tcsolid(f): Soil thermal conductivity for fibric (W/mK)
current value: 0.25, range: 0.005-0.5

*Ekici et al. 2015,Jiang et al. 2015, O’Donnell et al. 2009*

#### tcsolid(h): Soil thermal conductivity for humic (W/mK)
current value: 0.25, range: 0.02-2.0

*Ekici et al. 2015,Jiang et al. 2015, O’Donnell et al. 2009*

---------------------------------------------------------
#### porosity(m): porosity for moss layers (m3/m3)
current value: 0.98, range: 0.85-0.99

*O’Donnell et al. 2009*

#### porosity(f): porosity for fibric layers  (m3/m3)
current value: 0.95, range: 0.85-0.99

*O’Donnell et al. 2009*

#### porosity(h): porosity for humic layers  (m3/m3)
current value: 0.8, range: 0.7-0.9

*O’Donnell et al. 2009*

--------------------------------------------------------
#### bulkden(m): bulk density for moss (g/m3)
current value: 25,000, range: 10,000 - 80,000

*Tuomi et al. 2020, Rodionov et al. 2007, O’Donnell et al. 2009*

#### bulkden(f): bulk density for fibric (g/m3)
current value: 51,000, range: 20,000 - 200,000

*Tuomi et al. 2020, Rodionov et al. 2007, O’Donnell et al. 2009*

#### bulkden(h): bulk density for humic (g/m3)
current value: 176,000, range: 100,000 - 800,000

*Tuomi et al. 2020, Rodionov et al. 2007, O’Donnell et al. 2009*

--------------------------------------------------------
#### hksat(m): hydraulic conductivity at saturation for moss (mm/s)
current value: 0.15, range: 0.0002 - 30

*Ekici et al. 2015, Letts et al. 2000, Liu et al. 2019*

#### hksat(f): hydraulic conductivity at saturation for fibric (mm/s)
current value: 0.28, range: 0.0002 - 30

*Ekici et al. 2015, Letts et al. 2000, Liu et al. 2019*

#### hksat(h): hydraulic conductivity at saturation for humic (mm/s)
current value: 0.002, range: 0.00004 - 2.01

*Ekici et al. 2015, Letts et al. 2000, Liu et al. 2019*

---------------------------------------------------------
#### nfactor(s): Summer nfactor
current value: 1.5, range: 0.2-2.0

*Kade et al. 2006, Klene et al. 2001*

#### nfactor(w): Winter nfactor
current value: 1.0, range: 0.4-1.0

*Kade et al. 2006, Klene et al. 2001*

---------------------------------------------------------
#### snwalbmax
current value: 0.8, range: 0.7-0.85

*Te Beest et al. 2016, Petzold et al. 1975, Loranty et al. 2011*

#### snwalbmin
current value: 0.4, range: 0.4-0.6

*Te Beest et al. 2016, Petzold et al. 1975, Loranty et al. 2011*

---------------------------------------------------------
### cmt_dimground.txt
---------------------------------------------------------
#### snwdenmax (kg/m3)
current value: 250, range: 100-800

*Domine et al. 2016, Gerland et al. 1999, Muskett 2012*

#### snwdennew (kg/m3)
current value: 50, range: 10-250

*Domine et al. 2016, Gerland et al. 1999, Muskett 2012*


### cmt_bgcsoil.txt
---------------------------------------------------------
#### rhq10
current value: 2, range: 1.6-2.4
#### rhq10_w
current value: 2, range: 0.85-0.99

In [4]:
driver = sa.SensitivityDriver(work_dir = work_dir, clean=True)

work_dir='/data/workflows/US-Prr_SWC_SA'
opt_run_setup='--tr-yrs=121 --sp-yrs=300 --eq-yrs=500 '

driver.site = '/data/input-catalog/caribou-poker_merged/'
driver.opt_run_setup = opt_run_setup
driver.PXx ='1'
driver.PXy='0'

params = ['hksat(m)','hksat(f)','hksat(h)',
          'tcsolid(m)', 'tcsolid(f)', 'tcsolid(h)',
          'porosity(m)', 'porosity(f)', 'porosity(h)',
          'nfactor(s)', 'nfactor(w)',
          'rhq10']#, 'rhq10_w']
percent_diffs = [0.1, 0.1, 0.1,
                 0.1, 0.1, 0.1,
                 0.1, 0.1, 0.1,
                 0.1, 0.1,
                 0.1]#, 0.1]
bounds = [[0.0002, 30], [0.0002, 30], [0.00004, 2.01],
          [0.005, 0.5], [0.005, 0.5], [0.02, 2.0],
          [0.85, 0.99], [0.85, 0.99], [0.7, 0.9],
          [0.2, 2.0], [0.4, 1.0],
          [1.6, 2.4]]#, [1.6, 2.4]]
driver.logparams = [1, 1, 1,
                    1, 1, 1,
                    0, 0, 0,
                    0, 0,
                    0]#, 0]

driver.outputs = [
      { 'name': 'GPP', 'type': 'flux'},
      { 'name': 'RH','type': 'flux'},
      { 'name': 'LWCLAYER','type': 'layer'},
      { 'name': 'VWCLAYER','type': 'layer'},
      { 'name': 'TLAYER','type': 'layer'},
      { 'name': 'LAYERDEPTH','type': 'layer'},
      { 'name': 'LAYERDZ','type': 'layer'},
      { 'name': 'LAYERTYPE','type': 'layer'},
    ]

driver.design_experiment(Nsamples = 5, cmtnum = 13, params = params, percent_diffs = percent_diffs,
                         bounds=bounds, pftnums = [None]*len(params), sampling_method='uniform')


In [5]:
driver.setup_multi()

In [6]:
driver.sample_matrix

,hksat(m),hksat(f),hksat(h),tcsolid(m),tcsolid(f),tcsolid(h),porosity(m),porosity(f),porosity(h),nfactor(s),nfactor(w),rhq10
0,0.020552,0.128847,0.170800,0.143544,0.266181,0.089412,0.858132,0.971265,0.820223,1.474531,0.412351,2.375928
1,0.005076,0.001073,0.000344,0.174386,0.088220,0.575761,0.910472,0.890772,0.822371,0.451089,0.575287,1.893089
2,3.896287,2.839715,0.000042,0.007032,0.022949,0.376830,0.935056,0.873873,0.713010,1.907994,0.979379,2.246718
3,0.014048,0.000486,0.272681,0.026056,0.006700,1.189749,0.854814,0.977305,0.751756,1.392540,0.587027,2.016054
4,0.005691,25.659420,0.084157,0.008525,0.020938,0.175979,0.933706,0.979062,0.717699,0.552769,0.427136,1.860264


In [7]:
driver.run_all_samples()

[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000004/config/config.js[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000001/config/config.js[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000000/config/config.js[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000002/config/config.js[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000003/config/config.js












In [8]:
driver.plot_sensitivity_matrix()

AttributeError: 'SensitivityDriver' object has no attribute 'plot_sensitivity_matrix'

In [ ]:
print(LWCLAYER)

In [ ]:
def seasonal_profile(VAR, depth, thickness, time_range, months, z):
    LWCLAYER

In [ ]:
def seasonal_profile(VAR, depth, thickness, time_range, months, z):
    '''
    VAR : variable dataframe (i.e. TLAYER - temperature by layer)
    depth : associated LAYERDEPTH dataframe
    thickness : associated LAYERDZ dataframe
    time_range : time period to be calculated over -  e.g. ['2011-01-01','2021-01-01']
    months : list of strings - months included in calculation - ['Jan', 'Feb', 'Dec'] (e.g winter season)
    z : array for depth required for interpolation to the same depth profile 
         (typically, np.linspace(min(depth), max(depth), resolution)
    '''
    month_range = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    startyr, endyr = time_range[0], time_range[1]

    #Limiting by time range    
    range_series = VAR[startyr:endyr]
    LD = depth[startyr:endyr]
    LZ = thickness[startyr:endyr]

    #Averaging by layer for months in month_range
    range_series = range_series[range_series.index.month.isin([i+1 for i, e in enumerate(month_range) if e in months])]
    LD = (LD[LD.index.month.isin([i+1 for i, e in enumerate(month_range) if e in months])]).mean()
    LZ = (LZ[LZ.index.month.isin([i+1 for i, e in enumerate(month_range) if e in months])]).mean()

    #Creating lists for interpolation
    mean = range_series.mean().values.tolist()    
    maxi = range_series.max().values.tolist()
    mini = range_series.min().values.tolist()
    std =  range_series.std().values.tolist()

    #Creating list with z position at the center of each layer
    print(LD)
    print(LZ)
    zp = (LD + LZ/2).values.tolist()
    print(zp)
    print(z)
    print(mean)
    
    #Indexes of depths with fill values
    indexes = [i for i,v in enumerate(zp) if v < 0]

    #Removing indexes with fill values
    for index in sorted(indexes, reverse=True):
        del zp[index]
        del mean[index]
        del mini[index]
        del maxi[index]
        del std[index]

    #Interpolating seasonal mean, min, max, and standard deviation
    interp_mean=np.interp(z,zp,mean)
    interp_mini=np.interp(z,zp,mini)
    interp_maxi=np.interp(z,zp,maxi)
    interp_std=np.interp(z,zp,std)
    
    return z, interp_mean, interp_std, interp_mini, interp_maxi

In [ ]:
ou.load_trsc_dataframe

In [ ]:
LWCLAYER = ou.load_trsc_dataframe(var ='LWCLAYER', timeres='monthly', px_y=0, px_x=1, fileprefix='/data/workflows/US-Prr_SWC_SA/sample_000000000/output/')[0]
TLAYER = ou.load_trsc_dataframe(var ='TLAYER', timeres='monthly', px_y=0, px_x=1, fileprefix='/data/workflows/US-Prr_SWC_SA/sample_000000000/output/')[0]
LAYERDEPTH = ou.load_trsc_dataframe(var ='LAYERDEPTH', timeres='monthly', px_y=0, px_x=1, fileprefix='/data/workflows/US-Prr_SWC_SA/sample_000000000/output/')[0]
LAYERDZ = ou.load_trsc_dataframe(var ='LAYERDZ', timeres='monthly', px_y=0, px_x=1, fileprefix='/data/workflows/US-Prr_SWC_SA/sample_000000000/output/')[0]

In [ ]:
LAYERTYPE = ou.load_trsc_dataframe(var ='LAYERTYPE', timeres='monthly', px_y=0, px_x=1, fileprefix='/data/workflows/US-Prr_SWC_SA/sample_000000000/output/')[0]

In [ ]:
pf_obs_path = '/data/comparison_data/US-Prr-monthly.csv'
pf_obs= pd.read_csv(pf_obs_path, parse_dates=['m_y'])
pf_obs['Month'] = pf_obs['m_y'].dt.month
pf_obs = pf_obs.set_index('m_y')
swc = pf_obs[['SWC_1_1_1', 'SWC_1_2_1', 'SWC_1_3_1', 'SWC_1_4_1', 'SWC_1_5_1']]/100
#swc['Month'] = pf_obs['Month']
swc_depths = pd.DataFrame({'SWC_1_1_1':[0.05]*len(pf_obs), 'SWC_1_2_1':[0.1]*len(pf_obs), 'SWC_1_3_1':[0.2]*len(pf_obs),
                          'SWC_1_4_1':[0.3]*len(pf_obs), 'SWC_1_5_1':[0.4]*len(pf_obs)})
swc_depths = swc_depths.set_index(swc.index)
swc_thickness = pd.DataFrame({'SWC_1_1_1':[0.75]*len(pf_obs), 'SWC_1_2_1':[0.75]*len(pf_obs), 'SWC_1_3_1':[0.1]*len(pf_obs),
                          'SWC_1_4_1':[0.1]*len(pf_obs), 'SWC_1_5_1':[0.1]*len(pf_obs)})
swc_thickness = swc_thickness.set_index(swc.index)

In [ ]:
LAYERDZ

In [ ]:
swc

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()

z, mean, std, mn, mx = seasonal_profile(
                                        TLAYER, LAYERDEPTH, LAYERDZ, ['1901-01-01', '1902-12-01'], 
                                        ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'],
                                        np.linspace(min(LAYERDEPTH), max(LAYERDEPTH), 100)
                                        )


ax.plot(mean, z, label="annual mean")
ax.fill_betweenx(z, mn, mx, alpha=0.2, label="annual range")
ax.plot(np.zeros(len(z)), z, 'k--', alpha=0.5) #Zero degree line

plt.legend(loc="lower right", fontsize=12)

ax.xaxis.tick_top()
ax.set_xlabel(f"Temperature [$^\circ$C]", fontsize=14)
ax.xaxis.set_label_position('top') 

ax.set_ylabel(f"Depth [m]", fontsize=14)
plt.ylim(0,1)
ax.invert_yaxis()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()

z, mean, std, mn, mx = seasonal_profile(
                                        LWCLAYER, LAYERDEPTH, LAYERDZ, ['2010-01-01', '2021-12-01'], 
                                        ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'],
                                        np.linspace(min(LAYERDEPTH), max(LAYERDEPTH), 100)
                                        )


ax.plot(mean, z, label="annual mean")
ax.fill_betweenx(z, mn, mx, alpha=0.2, label="annual range")
ax.plot(np.zeros(len(z)), z, 'k--', alpha=0.5) #Zero degree line

plt.legend(loc="lower right", fontsize=12)

ax.xaxis.tick_top()
ax.set_xlabel(f"Temperature [$^\circ$C]", fontsize=14)
ax.xaxis.set_label_position('top') 

ax.set_ylabel(f"Depth [m]", fontsize=14)
plt.ylim(0,1)
ax.invert_yaxis()


plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()

z, mean, std, mn, mx = seasonal_profile(
                                        swc, swc_depths, swc_thickness, ['2010-01-01', '2021-12-01'], 
                                        ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'],
                                        np.linspace(np.nanmin(swc_depths), np.nanmax(swc_depths), 100)
                                        )


ax.plot(mean, z, label="annual mean")
#ax.fill_betweenx(z, mn, mx, alpha=0.2, label="annual range")
ax.plot(np.zeros(len(z)), z, 'k--', alpha=0.5) #Zero degree line

plt.legend(loc="lower right", fontsize=12)

ax.xaxis.tick_top()
ax.set_xlabel(f"Temperature [$^\circ$C]", fontsize=14)
ax.xaxis.set_label_position('top') 

ax.set_ylabel(f"Depth [m]", fontsize=14)
plt.ylim(0,1)
ax.invert_yaxis()


plt.show()

In [ ]:
print(mean)

In [ ]:
len(swc_groupby)

In [ ]:
swc_groupby['depth'] = [-.5, -.1, -.2, -.3, -.4]

In [ ]:
sns.lineplot(data=swc_groupby, x='1', y='depth')